# Script to calculate SEM model for the WMHOB project

In [2]:
library(ppcor)
#install.packages('lmPerm')
library(lmPerm)
library(psych)
library(car)
library(ggplot2)
library(lme4)
library(mediation)
library(caret)
library(gbm)
library(party)
library(Metrics)
#install.packages("wesanderson")
library(wesanderson)
library(gtools)
library(interactions)
#install.packages('ggstance')
library(ggstance)
#install.packages('elasticnet')
library(elasticnet)
#install.packages('kernlab')
library(kernlab)
#install.packages('e1071')
library(e1071)
#install.packages('MatchIt')
library(MatchIt)
library(dplyr)
library(ukbtools)
library(mediation)
library(lavaan)
library(lavaanPlot)
library(plotly)
library(MVN)
#install.packages('MissMech')
library(semTools)
library(MissMech)

# Load data

In [3]:
my_ukb_data <- ukb_df("ukb38276", path = "/dagher/dagher11/filip/OBAL/UKBB_data/")
load("/home/bic/fmorys/Projects/UKBB_all/di.Rda")
colnames(di)=c('eid',colnames(di)[2:length(di)])

### Exclude outliers in the imaging dataset (n=22k)
di$bipolar_dis <- NA
di$bipolar_dis[di$bipolar_disorder_status_f20122_0_0=="Bipolar Type I (Mania)"] <- 1
di$bipolar_dis[di$bipolar_disorder_status_f20122_0_0=="Bipolar Type II (Hypomania)"] <- 1

#Personality Disorders
di$personality_dis <- NA
for (i in as.numeric(grep('mental_health_problems_ever_diagnosed_by_a_professional', 
                          colnames(di)))) { #Columns with mental illness
di$personality_dis[di[i]=="A personality disorder"] <- 1
}

#Mental health issues
di$schizophrenia <- NA
for (i in as.numeric(grep('mental_health_problems_ever_diagnosed_by_a_professional', 
                          colnames(di)))) { #Columns with mental illness
di$schizophrenia[di[i]=="Schizophrenia"] <- 1
di$schizophrenia[di[i]=="Depression"] <- 1
di$schizophrenia[di[i]=="Mania, hypomania, bipolar or manic-depression"] <- 1
di$schizophrenia[di[i]=="Bulimia nervosa"] <- 1
di$schizophrenia[di[i]=="Anorexia nervosa"] <- 1
di$schizophrenia[di[i]=="Attention deficit or attention deficit and hyperactivity disorder (ADD/ADHD)"] <- 1
di$schizophrenia[di[i]=="Panic attacks"] <- 1
}


# Exclude participants who have diabetes
di$vascular_heart_diagnoses <- NA
for (i in as.numeric(grep('diabetes_diagnosed_by_doctor', 
                          colnames(di)))) { #Columns with vascular diseases
di$diabetes[di[i] =="Yes"] <- 1
}

# Exclude participants who had a heart attack, angina or stroke
di$vascular_heart_diagnoses <- NA
for (i in as.numeric(grep('vascularheart_problems_diagnosed_by_doctor', 
                          colnames(di)))) { #Columns with vascular diseases
di$vascular_heart_diagnoses[di[i] =="High blood pressure"] <- 1
}



#Dementia
for (i in as.numeric(grep('noncancer_illness_code_selfreported', 
                          colnames(di)))) { #Columns with noncancer illness for all timepoints
di$neurological_disorder[di[i] == "1263"] <- 1 #Dementia or Alzheimer's
di$neurological_disorder[di[i] == "1262"] <- 1 #Parkinson Disease
di$neurological_disorder[di[i] == "1258"] <- 1 #Chronic degenerative neurological
di$neurological_disorder[di[i] == "1256"] <- 1 #Guillain Barré
di$neurological_disorder[di[i] == "1261"] <- 1 #Multiple slerosis
di$neurological_disorder[di[i] == "1297"] <- 1 #Other demyenilating disease
di$neurological_disorder[di[i] == "1081"] <- 1 #Stroke
di$neurological_disorder[di[i] == "1032"] <- 1 #Brain cancer
di$neurological_disorder[di[i] == "1491"] <- 1 #Brain hemorrhage
di$neurological_disorder[di[i] == "1245"] <- 1 #Brain/ intracranial abcess
di$neurological_disorder[di[i] == "1425"] <- 1 #Cerebral aneurism
di$neurological_disorder[di[i] == "1433"] <- 1 #Cerebral palsy
di$neurological_disorder[di[i] == "1256"] <- 1 #Encephalitis
di$neurological_disorder[di[i] == "1264"] <- 1 #Epilepsy
di$neurological_disorder[di[i] == "1266"] <- 1 #Head injury
di$neurological_disorder[di[i] == "1244"] <- 1 #Infections of the nervous system
di$neurological_disorder[di[i] == "1583"] <- 1 #Ischaemic stroke
di$neurological_disorder[di[i] == "1031"] <- 1 #Meningeal cancer
di$neurological_disorder[di[i] == "1659"] <- 1 #Meningioma (benign)
di$neurological_disorder[di[i] == "1247"] <- 1 #Meningitis
di$neurological_disorder[di[i] == "1259"] <- 1 #Motor neuron disease
di$neurological_disorder[di[i] == "1240"] <- 1 #Neurological injury/trauma
di$neurological_disorder[di[i] == "1524"] <- 1 #Spina bifida
di$neurological_disorder[di[i] == "1083"] <- 1 #Subdural hematoma
di$neurological_disorder[di[i] == "1086"] <- 1 #Subarachnoid haemorrage
di$neurological_disorder[di[i] == "1082"] <- 1 #Transient ischaemic attack
}

#If BMI < 18.5 (Re-do this, cause it's not usually done)
#di$underweight1[di$body_mass_index_bmi_f21001_0_0 < 18.5] <- 1
#di$underweight2[di$body_mass_index_bmi_f21001_1_0 < 18.5] <- 1
di$underweight3[di$body_mass_index_bmi_f21001_2_0 < 18.5] <- 1

#Exclude participants if no neuroimaging data, bipolar_dis == 1; personality_dis == 1; schizophrenia == 1; vascular_heart_diagnoses ==1

di$excluded <- NA
di$excluded[di$bipolar_dis ==1] <- 1
di$excluded[di$personality_dis == 1] <- 1
di$excluded[di$schizophrenia == 1] <- 1
#di$excluded[di$vascular_heart_diagnoses ==1] <- 1
#di$excluded[di$underweight1 ==1] <- 1
#di$excluded[di$underweight2 ==1] <- 1
di$excluded[di$underweight3 == 1] <- 1
di$excluded[di$neurological_disorder ==1] <- 1
#di$excluded[di$diabetes ==1] <- 1

di$included<-car::recode(di$excluded, "1='excluded'; else='included'")


#Select subjects if included ==1
di_excluded <- subset(di, included=="included")

ukbb_all=merge(my_ukb_data, di_excluded, by.x='eid')

nrow(ukbb_all)

ukbb_all$WMH_logscaled=log(ukbb_all$total_volume_of_white_matter_hyperintensities_from_t1_and_t2_flair_images_f25781_2_0*
                    ukbb_all$volumetric_scaling_from_t1_head_image_to_standard_space_f25000_2_0)
ukbb_all$logBMI3=log(ukbb_all$body_mass_index_bmi_f21001_2_0)
ukbb_all$WHR=ukbb_all$waist_circumference_f48_2_0/ukbb_all$hip_circumference_f49_2_0

## Merge with white matter integrity variables
#ukb_WM <- ukb_df('ukb35375', path = "/dagher/dagher11/filip/OBAL/UKBB_data/")
#ukbb_all=merge(ukbb_all,ukb_WM,by='eid',all.x=TRUE)

ukbb_all$Hypertension=0
ukbb_all$Hypertension[ukbb_all$vascular_heart_diagnoses=='1']=1
ukbb_all$Diabetes=0
ukbb_all$Diabetes[ukbb_all$diabetes=='1']=1



In [4]:
UKB_Blood <- ukb_df("ukb41240", path = "/dagher/dagher11/filip/OBAL/UKBB_data/")

In [5]:
ukbb_all=merge(ukbb_all,UKB_Blood, by='eid', all.x=TRUE)

In [6]:
ukbb_all$MeanThickness=scale(rowMeans(ukbb_all[grep('thickness',colnames(ukbb_all))]))

## Convert cognitive variables

In [7]:
ukbb_all$Mean_pairs=scale(rowMeans(ukbb_all[274:275], na.rm=TRUE, dims=1))
ukbb_all$RT=scale(rowMeans(ukbb_all[418:429], na.rm=TRUE, dims=1))
ukbb_all$Prospective_memory<-factor(car::recode(ukbb_all$prospective_memory_result_f20018_2_0, 
                                         "'Correct recall on first attempt'='1'; 
                                        'Correct recall on second attempt'='2';
                                        'Instruction not recalled, either skipped or incorrect'='3'"), ordered=TRUE)
ukbb_cog=ukbb_all


## Standardise brain volume variables

In [8]:
for (i in c(2287:2300)) {
    ukbb_all[[i]]=scale(ukbb_all[[i]]/ukbb_all[2116])
    }

In [9]:
for (i in c(58:88, 120:150)) {
    ukbb_all[[i]]=scale(ukbb_all[[i]])
    }

In [10]:
for (i in c(89:119, 151:181)) {
    ukbb_all[[i]]=scale(ukbb_all[[i]]/ukbb_all$volume_of_estimatedtotalintracranial_whole_brain_f26521_2_0)
    }

## Scale and log-transform variables

In [12]:
ukbb_all$Sex=as.factor(ukbb_all$sex_f31_0_0)
ukbb_all$DigitSpan=scale(ukbb_all$maximum_digits_remembered_correctly_f4282_2_0)
ukbb_all$FluidIntelligence=scale(ukbb_all$fluid_intelligence_score_f20016_2_0)
ukbb_all$Puzzles=scale(ukbb_all$number_of_puzzles_correct_f21004_2_0)
ukbb_all$WHRscaled=scale(ukbb_all$WHR)
ukbb_all$CRP_scaled=(log(ukbb_all$creactive_protein_f30710_0_0))
ukbb_all$SBP=scale(rowMeans(data.frame(ukbb_all$systolic_blood_pressure_automated_reading_f4080_2_1, ukbb_all$systolic_blood_pressure_automated_reading_f4080_2_0)))
ukbb_all$TG=scale(log(ukbb_all$triglycerides_f30870_0_0))
ukbb_all$HDL=scale(log(ukbb_all$hdl_cholesterol_f30760_0_0))

In [13]:
ukbb_all$Age_scaled=scale(ukbb_all$age_when_attended_assessment_centre_f21003_0_0)

In [14]:
ukbb_all$BMI3=scale(log(ukbb_all$body_mass_index_bmi_f21001_2_0))

## Convert education variable - not used further

In [46]:
ukbb_all$Edu1<-car::recode(ukbb_all$qualifications_f6138_0_0, "'College or University degree'=1;
'A levels/AS levels or equivalent'=2; 'O levels/GCSEs or equivalent'=3; 
'CSEs or equivalent'=4; 'NVQ or HND or HNC or equivalent'=5; 'Other professional qualifications eg':' nursing, teaching'=6;
'Prefer not to answer'=7; else=NA")
ukbb_all$Edu1=as.numeric(ukbb_all$Edu1)

## Create one dataframe with all variables for the SEM and remove outliers

In [15]:
Lavaan_SEM=select(ukbb_all, Sex, Prospective_memory, Hypertension, Diabetes,
                  Age_scaled, 
                  mean_thickness_of_caudalanteriorcingulate_left_hemisphere_f27174_2_0,
                    mean_thickness_of_caudalmiddlefrontal_left_hemisphere_f27175_2_0,
                    mean_thickness_of_cuneus_left_hemisphere_f27176_2_0,
                    mean_thickness_of_entorhinal_left_hemisphere_f27177_2_0,
                    mean_thickness_of_fusiform_left_hemisphere_f27178_2_0,
                    mean_thickness_of_inferiorparietal_left_hemisphere_f27179_2_0,
                    mean_thickness_of_inferiortemporal_left_hemisphere_f27180_2_0,
                    mean_thickness_of_isthmuscingulate_left_hemisphere_f27181_2_0,
                    mean_thickness_of_lateraloccipital_left_hemisphere_f27182_2_0,
                    mean_thickness_of_lateralorbitofrontal_left_hemisphere_f27183_2_0,
                    mean_thickness_of_lingual_left_hemisphere_f27184_2_0,
                    mean_thickness_of_medialorbitofrontal_left_hemisphere_f27185_2_0,
                    mean_thickness_of_middletemporal_left_hemisphere_f27186_2_0,
                    mean_thickness_of_parahippocampal_left_hemisphere_f27187_2_0,
                    mean_thickness_of_paracentral_left_hemisphere_f27188_2_0,
                    mean_thickness_of_parsopercularis_left_hemisphere_f27189_2_0,
                    mean_thickness_of_parsorbitalis_left_hemisphere_f27190_2_0,
                    mean_thickness_of_parstriangularis_left_hemisphere_f27191_2_0,
                    mean_thickness_of_pericalcarine_left_hemisphere_f27192_2_0,
                    mean_thickness_of_postcentral_left_hemisphere_f27193_2_0,
                    mean_thickness_of_posteriorcingulate_left_hemisphere_f27194_2_0,
                    mean_thickness_of_precentral_left_hemisphere_f27195_2_0,
                    mean_thickness_of_precuneus_left_hemisphere_f27196_2_0,
                    mean_thickness_of_rostralanteriorcingulate_left_hemisphere_f27197_2_0,
                    mean_thickness_of_rostralmiddlefrontal_left_hemisphere_f27198_2_0,
                    mean_thickness_of_superiorfrontal_left_hemisphere_f27199_2_0,
                    mean_thickness_of_superiorparietal_left_hemisphere_f27200_2_0,
                    mean_thickness_of_superiortemporal_left_hemisphere_f27201_2_0,
                    mean_thickness_of_supramarginal_left_hemisphere_f27202_2_0,
                    mean_thickness_of_transversetemporal_left_hemisphere_f27203_2_0,
                    mean_thickness_of_insula_left_hemisphere_f27204_2_0,
                    mean_thickness_of_caudalanteriorcingulate_right_hemisphere_f27267_2_0,
                    mean_thickness_of_caudalmiddlefrontal_right_hemisphere_f27268_2_0,
                    mean_thickness_of_cuneus_right_hemisphere_f27269_2_0,
                    mean_thickness_of_entorhinal_right_hemisphere_f27270_2_0,
                    mean_thickness_of_fusiform_right_hemisphere_f27271_2_0,
                    mean_thickness_of_inferiorparietal_right_hemisphere_f27272_2_0,
                    mean_thickness_of_inferiortemporal_right_hemisphere_f27273_2_0,
                    mean_thickness_of_isthmuscingulate_right_hemisphere_f27274_2_0,
                    mean_thickness_of_lateraloccipital_right_hemisphere_f27275_2_0,
                    mean_thickness_of_lateralorbitofrontal_right_hemisphere_f27276_2_0,
                    mean_thickness_of_lingual_right_hemisphere_f27277_2_0,
                    mean_thickness_of_medialorbitofrontal_right_hemisphere_f27278_2_0,
                    mean_thickness_of_middletemporal_right_hemisphere_f27279_2_0,
                    mean_thickness_of_parahippocampal_right_hemisphere_f27280_2_0,
                    mean_thickness_of_paracentral_right_hemisphere_f27281_2_0,
                    mean_thickness_of_parsopercularis_right_hemisphere_f27282_2_0,
                    mean_thickness_of_parsorbitalis_right_hemisphere_f27283_2_0,
                    mean_thickness_of_parstriangularis_right_hemisphere_f27284_2_0,
                    mean_thickness_of_pericalcarine_right_hemisphere_f27285_2_0,
                    mean_thickness_of_postcentral_right_hemisphere_f27286_2_0,
                    mean_thickness_of_posteriorcingulate_right_hemisphere_f27287_2_0,
                    mean_thickness_of_precentral_right_hemisphere_f27288_2_0,
                    mean_thickness_of_precuneus_right_hemisphere_f27289_2_0,
                    mean_thickness_of_rostralanteriorcingulate_right_hemisphere_f27290_2_0,
                    mean_thickness_of_rostralmiddlefrontal_right_hemisphere_f27291_2_0,
                    mean_thickness_of_superiorfrontal_right_hemisphere_f27292_2_0,
                    mean_thickness_of_superiorparietal_right_hemisphere_f27293_2_0,
                    mean_thickness_of_superiortemporal_right_hemisphere_f27294_2_0,
                    mean_thickness_of_supramarginal_right_hemisphere_f27295_2_0,
                    mean_thickness_of_transversetemporal_right_hemisphere_f27296_2_0,
                    mean_thickness_of_insula_right_hemisphere_f27297_2_0,
                    volume_of_caudalanteriorcingulate_left_hemisphere_f27205_2_0,
                    volume_of_caudalmiddlefrontal_left_hemisphere_f27206_2_0,
                    volume_of_cuneus_left_hemisphere_f27207_2_0,
                    volume_of_entorhinal_left_hemisphere_f27208_2_0,
                    volume_of_fusiform_left_hemisphere_f27209_2_0,
                    volume_of_inferiorparietal_left_hemisphere_f27210_2_0,
                    volume_of_inferiortemporal_left_hemisphere_f27211_2_0,
                    volume_of_isthmuscingulate_left_hemisphere_f27212_2_0,
                    volume_of_lateraloccipital_left_hemisphere_f27213_2_0,
                    volume_of_lateralorbitofrontal_left_hemisphere_f27214_2_0,
                    volume_of_lingual_left_hemisphere_f27215_2_0,
                    volume_of_medialorbitofrontal_left_hemisphere_f27216_2_0,
                    volume_of_middletemporal_left_hemisphere_f27217_2_0,
                    volume_of_parahippocampal_left_hemisphere_f27218_2_0,
                    volume_of_paracentral_left_hemisphere_f27219_2_0,
                    volume_of_parsopercularis_left_hemisphere_f27220_2_0,
                    volume_of_parsorbitalis_left_hemisphere_f27221_2_0,
                    volume_of_parstriangularis_left_hemisphere_f27222_2_0,
                    volume_of_pericalcarine_left_hemisphere_f27223_2_0,
                    volume_of_postcentral_left_hemisphere_f27224_2_0,
                    volume_of_posteriorcingulate_left_hemisphere_f27225_2_0,
                    volume_of_precentral_left_hemisphere_f27226_2_0,
                    volume_of_precuneus_left_hemisphere_f27227_2_0,
                    volume_of_rostralanteriorcingulate_left_hemisphere_f27228_2_0,
                    volume_of_rostralmiddlefrontal_left_hemisphere_f27229_2_0,
                    volume_of_superiorfrontal_left_hemisphere_f27230_2_0,
                    volume_of_superiorparietal_left_hemisphere_f27231_2_0,
                    volume_of_superiortemporal_left_hemisphere_f27232_2_0,
                    volume_of_supramarginal_left_hemisphere_f27233_2_0,
                    volume_of_transversetemporal_left_hemisphere_f27234_2_0,
                    volume_of_insula_left_hemisphere_f27235_2_0,
                    volume_of_caudalanteriorcingulate_right_hemisphere_f27298_2_0,
                    volume_of_caudalmiddlefrontal_right_hemisphere_f27299_2_0,
                    volume_of_cuneus_right_hemisphere_f27300_2_0,
                    volume_of_entorhinal_right_hemisphere_f27301_2_0,
                    volume_of_fusiform_right_hemisphere_f27302_2_0,
                    volume_of_inferiorparietal_right_hemisphere_f27303_2_0,
                    volume_of_inferiortemporal_right_hemisphere_f27304_2_0,
                    volume_of_isthmuscingulate_right_hemisphere_f27305_2_0,
                    volume_of_lateraloccipital_right_hemisphere_f27306_2_0,
                    volume_of_lateralorbitofrontal_right_hemisphere_f27307_2_0,
                    volume_of_lingual_right_hemisphere_f27308_2_0,
                    volume_of_medialorbitofrontal_right_hemisphere_f27309_2_0,
                    volume_of_middletemporal_right_hemisphere_f27310_2_0,
                    volume_of_parahippocampal_right_hemisphere_f27311_2_0,
                    volume_of_paracentral_right_hemisphere_f27312_2_0,
                    volume_of_parsopercularis_right_hemisphere_f27313_2_0,
                    volume_of_parsorbitalis_right_hemisphere_f27314_2_0,
                    volume_of_parstriangularis_right_hemisphere_f27315_2_0,
                    volume_of_pericalcarine_right_hemisphere_f27316_2_0,
                    volume_of_postcentral_right_hemisphere_f27317_2_0,
                    volume_of_posteriorcingulate_right_hemisphere_f27318_2_0,
                    volume_of_precentral_right_hemisphere_f27319_2_0,
                    volume_of_precuneus_right_hemisphere_f27320_2_0,
                    volume_of_rostralanteriorcingulate_right_hemisphere_f27321_2_0,
                    volume_of_rostralmiddlefrontal_right_hemisphere_f27322_2_0,
                    volume_of_superiorfrontal_right_hemisphere_f27323_2_0,
                    volume_of_superiorparietal_right_hemisphere_f27324_2_0,
                    volume_of_superiortemporal_right_hemisphere_f27325_2_0,
                    volume_of_supramarginal_right_hemisphere_f27326_2_0,
                    volume_of_transversetemporal_right_hemisphere_f27327_2_0,
                    volume_of_insula_right_hemisphere_f27328_2_0, 
                    volume_of_grey_matter_in_thalamus_left_f25878_2_0,
                    volume_of_grey_matter_in_thalamus_right_f25879_2_0,
                    volume_of_grey_matter_in_caudate_left_f25880_2_0,
                    volume_of_grey_matter_in_caudate_right_f25881_2_0,
                    volume_of_grey_matter_in_putamen_left_f25882_2_0,
                    volume_of_grey_matter_in_putamen_right_f25883_2_0,
                    volume_of_grey_matter_in_pallidum_left_f25884_2_0,
                    volume_of_grey_matter_in_pallidum_right_f25885_2_0,
                    volume_of_grey_matter_in_hippocampus_left_f25886_2_0,
                    volume_of_grey_matter_in_hippocampus_right_f25887_2_0,
                    volume_of_grey_matter_in_amygdala_left_f25888_2_0,
                    volume_of_grey_matter_in_amygdala_right_f25889_2_0,
                    volume_of_grey_matter_in_ventral_striatum_left_f25890_2_0,
                    volume_of_grey_matter_in_ventral_striatum_right_f25891_2_0,
                    Mean_pairs, RT,
                    FluidIntelligence, DigitSpan, Puzzles, WHRscaled, BMI3, CRP_scaled, WMH_logscaled, TG, HDL, 
                  age_when_attended_assessment_centre_f21003_2_0,
                 body_mass_index_bmi_f21001_2_0, WHR, qualifications_f6138_2_0, age_when_attended_assessment_centre_f21003_2_0)



#Lavaan_SEM=Lavaan_SEM[complete.cases(Lavaan_SEM[1:(length(Lavaan_SEM))]),]
for (j in 7:(length(Lavaan_SEM)-2)) { # 
    Q3=as.numeric(quantile(Lavaan_SEM[j],0.75, na.rm=TRUE))
    Q1=as.numeric(quantile(Lavaan_SEM[j],0.25, na.rm=TRUE))
    upper=Q3+(2.2*(Q3-Q1))
    lower=Q1-(2.2*(Q3-Q1))
    Lavaan_SEM[j][Lavaan_SEM[j]<lower]=NA
    Lavaan_SEM[j][Lavaan_SEM[j]>upper]=NA
}

Lavaan_SEM=Lavaan_SEM[complete.cases(Lavaan_SEM$qualifications_f6138_2_0),]
nrow(Lavaan_SEM)

## Calculate characteristics

In [16]:
mean(Lavaan_SEM$age_when_attended_assessment_centre_f21003_2_0)
sd(Lavaan_SEM$age_when_attended_assessment_centre_f21003_2_0)
mean(Lavaan_SEM$body_mass_index_bmi_f21001_2_0, na.rm=TRUE)
sd(Lavaan_SEM$body_mass_index_bmi_f21001_2_0, na.rm=TRUE)
mean(Lavaan_SEM$WHR, na.rm=TRUE)
sd(Lavaan_SEM$WHR, na.rm=TRUE)
table(Lavaan_SEM$Sex)
table(Lavaan_SEM$Hypertension)
table(Lavaan_SEM$Diabetes)

## Model with triglycerides

In [17]:
WMH_SEM = '
#Latent variables
Cognition =~ Mean_pairs + RT + Prospective_memory +
          FluidIntelligence + DigitSpan + Puzzles
Obesity =~ BMI3 + WHRscaled
Inflammation =~ CRP_scaled
Cerebrovascular =~ WMH_logscaled
Dyslipidemia =~ TG + HDL
Thickness =~ mean_thickness_of_caudalanteriorcingulate_left_hemisphere_f27174_2_0+
            mean_thickness_of_caudalmiddlefrontal_left_hemisphere_f27175_2_0+
            mean_thickness_of_cuneus_left_hemisphere_f27176_2_0+
            mean_thickness_of_entorhinal_left_hemisphere_f27177_2_0+
            mean_thickness_of_fusiform_left_hemisphere_f27178_2_0+
            mean_thickness_of_inferiorparietal_left_hemisphere_f27179_2_0+
            mean_thickness_of_inferiortemporal_left_hemisphere_f27180_2_0+
            mean_thickness_of_isthmuscingulate_left_hemisphere_f27181_2_0+
            mean_thickness_of_lateraloccipital_left_hemisphere_f27182_2_0+
            mean_thickness_of_lateralorbitofrontal_left_hemisphere_f27183_2_0+
            mean_thickness_of_lingual_left_hemisphere_f27184_2_0+
            mean_thickness_of_medialorbitofrontal_left_hemisphere_f27185_2_0+
            mean_thickness_of_middletemporal_left_hemisphere_f27186_2_0+
            mean_thickness_of_parahippocampal_left_hemisphere_f27187_2_0+
            mean_thickness_of_paracentral_left_hemisphere_f27188_2_0+
            mean_thickness_of_parsopercularis_left_hemisphere_f27189_2_0+
            mean_thickness_of_parsorbitalis_left_hemisphere_f27190_2_0+
            mean_thickness_of_parstriangularis_left_hemisphere_f27191_2_0+
            mean_thickness_of_pericalcarine_left_hemisphere_f27192_2_0+
            mean_thickness_of_postcentral_left_hemisphere_f27193_2_0+
            mean_thickness_of_posteriorcingulate_left_hemisphere_f27194_2_0+
            mean_thickness_of_precentral_left_hemisphere_f27195_2_0+
            mean_thickness_of_precuneus_left_hemisphere_f27196_2_0+
            mean_thickness_of_rostralanteriorcingulate_left_hemisphere_f27197_2_0+
            mean_thickness_of_rostralmiddlefrontal_left_hemisphere_f27198_2_0+
            mean_thickness_of_superiorfrontal_left_hemisphere_f27199_2_0+
            mean_thickness_of_superiorparietal_left_hemisphere_f27200_2_0+
            mean_thickness_of_superiortemporal_left_hemisphere_f27201_2_0+
            mean_thickness_of_supramarginal_left_hemisphere_f27202_2_0+
            mean_thickness_of_transversetemporal_left_hemisphere_f27203_2_0+
            mean_thickness_of_insula_left_hemisphere_f27204_2_0+
            mean_thickness_of_caudalanteriorcingulate_right_hemisphere_f27267_2_0+
            mean_thickness_of_caudalmiddlefrontal_right_hemisphere_f27268_2_0+
            mean_thickness_of_cuneus_right_hemisphere_f27269_2_0+
            mean_thickness_of_entorhinal_right_hemisphere_f27270_2_0+
            mean_thickness_of_fusiform_right_hemisphere_f27271_2_0+
            mean_thickness_of_inferiorparietal_right_hemisphere_f27272_2_0+
            mean_thickness_of_inferiortemporal_right_hemisphere_f27273_2_0+
            mean_thickness_of_isthmuscingulate_right_hemisphere_f27274_2_0+
            mean_thickness_of_lateraloccipital_right_hemisphere_f27275_2_0+
            mean_thickness_of_lateralorbitofrontal_right_hemisphere_f27276_2_0+
            mean_thickness_of_lingual_right_hemisphere_f27277_2_0+
            mean_thickness_of_medialorbitofrontal_right_hemisphere_f27278_2_0+
            mean_thickness_of_middletemporal_right_hemisphere_f27279_2_0+
            mean_thickness_of_parahippocampal_right_hemisphere_f27280_2_0+
            mean_thickness_of_paracentral_right_hemisphere_f27281_2_0+
            mean_thickness_of_parsopercularis_right_hemisphere_f27282_2_0+
            mean_thickness_of_parsorbitalis_right_hemisphere_f27283_2_0+
            mean_thickness_of_parstriangularis_right_hemisphere_f27284_2_0+
            mean_thickness_of_pericalcarine_right_hemisphere_f27285_2_0+
            mean_thickness_of_postcentral_right_hemisphere_f27286_2_0+
            mean_thickness_of_posteriorcingulate_right_hemisphere_f27287_2_0+
            mean_thickness_of_precentral_right_hemisphere_f27288_2_0+
            mean_thickness_of_precuneus_right_hemisphere_f27289_2_0+
            mean_thickness_of_rostralanteriorcingulate_right_hemisphere_f27290_2_0+
            mean_thickness_of_rostralmiddlefrontal_right_hemisphere_f27291_2_0+
            mean_thickness_of_superiorfrontal_right_hemisphere_f27292_2_0+
            mean_thickness_of_superiorparietal_right_hemisphere_f27293_2_0+
            mean_thickness_of_superiortemporal_right_hemisphere_f27294_2_0+
            mean_thickness_of_supramarginal_right_hemisphere_f27295_2_0+
            mean_thickness_of_transversetemporal_right_hemisphere_f27296_2_0+
            mean_thickness_of_insula_right_hemisphere_f27297_2_0

Volume=~    volume_of_caudalanteriorcingulate_left_hemisphere_f27205_2_0+
            volume_of_caudalmiddlefrontal_left_hemisphere_f27206_2_0+
            volume_of_cuneus_left_hemisphere_f27207_2_0+
            volume_of_entorhinal_left_hemisphere_f27208_2_0+
            volume_of_fusiform_left_hemisphere_f27209_2_0+
            volume_of_inferiorparietal_left_hemisphere_f27210_2_0+
            volume_of_inferiortemporal_left_hemisphere_f27211_2_0+
            volume_of_isthmuscingulate_left_hemisphere_f27212_2_0+
            volume_of_lateraloccipital_left_hemisphere_f27213_2_0+
            volume_of_lateralorbitofrontal_left_hemisphere_f27214_2_0+
            volume_of_lingual_left_hemisphere_f27215_2_0+
            volume_of_medialorbitofrontal_left_hemisphere_f27216_2_0+
            volume_of_middletemporal_left_hemisphere_f27217_2_0+
            volume_of_parahippocampal_left_hemisphere_f27218_2_0+
            volume_of_paracentral_left_hemisphere_f27219_2_0+
            volume_of_parsopercularis_left_hemisphere_f27220_2_0+
            volume_of_parsorbitalis_left_hemisphere_f27221_2_0+
            volume_of_parstriangularis_left_hemisphere_f27222_2_0+
            volume_of_pericalcarine_left_hemisphere_f27223_2_0+
            volume_of_postcentral_left_hemisphere_f27224_2_0+
            volume_of_posteriorcingulate_left_hemisphere_f27225_2_0+
            volume_of_precentral_left_hemisphere_f27226_2_0+
            volume_of_precuneus_left_hemisphere_f27227_2_0+
            volume_of_rostralanteriorcingulate_left_hemisphere_f27228_2_0+
            volume_of_rostralmiddlefrontal_left_hemisphere_f27229_2_0+
            volume_of_superiorfrontal_left_hemisphere_f27230_2_0+
            volume_of_superiorparietal_left_hemisphere_f27231_2_0+
            volume_of_superiortemporal_left_hemisphere_f27232_2_0+
            volume_of_supramarginal_left_hemisphere_f27233_2_0+
            volume_of_transversetemporal_left_hemisphere_f27234_2_0+
            volume_of_insula_left_hemisphere_f27235_2_0+
            volume_of_caudalanteriorcingulate_right_hemisphere_f27298_2_0+
            volume_of_caudalmiddlefrontal_right_hemisphere_f27299_2_0+
            volume_of_cuneus_right_hemisphere_f27300_2_0+
            volume_of_entorhinal_right_hemisphere_f27301_2_0+
            volume_of_fusiform_right_hemisphere_f27302_2_0+
            volume_of_inferiorparietal_right_hemisphere_f27303_2_0+
            volume_of_inferiortemporal_right_hemisphere_f27304_2_0+
            volume_of_isthmuscingulate_right_hemisphere_f27305_2_0+
            volume_of_lateraloccipital_right_hemisphere_f27306_2_0+
            volume_of_lateralorbitofrontal_right_hemisphere_f27307_2_0+
            volume_of_lingual_right_hemisphere_f27308_2_0+
            volume_of_medialorbitofrontal_right_hemisphere_f27309_2_0+
            volume_of_middletemporal_right_hemisphere_f27310_2_0+
            volume_of_parahippocampal_right_hemisphere_f27311_2_0+
            volume_of_paracentral_right_hemisphere_f27312_2_0+
            volume_of_parsopercularis_right_hemisphere_f27313_2_0+
            volume_of_parsorbitalis_right_hemisphere_f27314_2_0+
            volume_of_parstriangularis_right_hemisphere_f27315_2_0+
            volume_of_pericalcarine_right_hemisphere_f27316_2_0+
            volume_of_postcentral_right_hemisphere_f27317_2_0+
            volume_of_posteriorcingulate_right_hemisphere_f27318_2_0+
            volume_of_precentral_right_hemisphere_f27319_2_0+
            volume_of_precuneus_right_hemisphere_f27320_2_0+
            volume_of_rostralanteriorcingulate_right_hemisphere_f27321_2_0+
            volume_of_rostralmiddlefrontal_right_hemisphere_f27322_2_0+
            volume_of_superiorfrontal_right_hemisphere_f27323_2_0+
            volume_of_superiorparietal_right_hemisphere_f27324_2_0+
            volume_of_superiortemporal_right_hemisphere_f27325_2_0+
            volume_of_supramarginal_right_hemisphere_f27326_2_0+
            volume_of_transversetemporal_right_hemisphere_f27327_2_0+
            volume_of_insula_right_hemisphere_f27328_2_0

Volume_sub =~ volume_of_grey_matter_in_thalamus_left_f25878_2_0+
            volume_of_grey_matter_in_thalamus_right_f25879_2_0+
            volume_of_grey_matter_in_caudate_left_f25880_2_0+
            volume_of_grey_matter_in_caudate_right_f25881_2_0+
            volume_of_grey_matter_in_putamen_left_f25882_2_0+
            volume_of_grey_matter_in_putamen_right_f25883_2_0+
            volume_of_grey_matter_in_pallidum_left_f25884_2_0+
            volume_of_grey_matter_in_pallidum_right_f25885_2_0+
            volume_of_grey_matter_in_hippocampus_left_f25886_2_0+
            volume_of_grey_matter_in_hippocampus_right_f25887_2_0+
            volume_of_grey_matter_in_amygdala_left_f25888_2_0+
            volume_of_grey_matter_in_amygdala_right_f25889_2_0+
            volume_of_grey_matter_in_ventral_striatum_left_f25890_2_0+
            volume_of_grey_matter_in_ventral_striatum_right_f25891_2_0


#regressions

Dyslipidemia ~ Obesity + Sex + Age_scaled
Hypertension ~ Obesity + Sex + Age_scaled
Diabetes ~  Obesity + Sex + Age_scaled
Inflammation ~ 
    Obesity + Sex + Age_scaled
Cerebrovascular ~ 
    Inflammation + Hypertension + Diabetes + Dyslipidemia + Sex + Age_scaled
Volume_sub ~
    Cerebrovascular + Sex + Age_scaled
Thickness ~
    Cerebrovascular + Sex + Age_scaled
Volume ~
    Cerebrovascular + Sex + Age_scaled
Cognition ~ 
    qualifications_f6138_2_0 + Obesity + Inflammation + Dyslipidemia + Thickness + Volume + Volume_sub + Cerebrovascular + Hypertension + Diabetes + Sex + Age_scaled

#covariance
Volume_sub ~~  Thickness
Volume ~~ Thickness
Volume_sub ~~ Volume
Inflammation ~~ Hypertension
Inflammation ~~ Diabetes
Inflammation ~~ Dyslipidemia
Hypertension ~~ Diabetes
Diabetes ~~ Dyslipidemia
Hypertension ~~ Dyslipidemia
Hypertension ~~ Volume_sub + Thickness + Volume
Diabetes ~~ Volume_sub + Thickness + Volume
Inflammation ~~ Volume_sub + Thickness + Volume
Dyslipidemia ~~ Volume_sub + Thickness + Volume
Obesity ~~ Volume_sub + Thickness + Volume
'



WMHOB_fit <- sem(WMH_SEM, data=Lavaan_SEM, missing='pairwise')
summary(WMHOB_fit, standardized=TRUE, fit.measures=TRUE)
#fitmeasures(WMHOB_fit) #fit indices

## Adjusted bonferroni correction of p-values according to https://www.tandfonline.com/doi/full/10.1080/10705511.2013.742385

In [65]:
alpha=0.05/(39^(1-(sqrt(mean(lavInspect(WMHOB_fit,"cor.all"))))))
a=c(0,0,0.65,0,0,0,0,0,0,0,0.021,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.037,0.068,0.064,0.0126,0,0,0.418,0.037,0,0)
a*(0.05/alpha)

In [24]:
a<-fitmeasures(WMHOB_fit)["chisq.scaling.factor"] 
rmsea.old<-fitmeasures(WMHOB_fit)["rmsea"] 
ci.old.l<-fitmeasures(WMHOB_fit)["rmsea.ci.lower"] 
ci.old.u<-fitmeasures(WMHOB_fit)["rmsea.ci.upper"] 
(rmsea.new<-sqrt(a)*rmsea.old) 
(ci.new.l<-sqrt(a)*ci.old.l) 
(ci.new.u<-sqrt(a)*ci.old.u) 
cfi.old<-fitmeasures(WMHOB_fit)["cfi"] 
a.baseline<-fitmeasures(WMHOB_fit)["baseline.chisq.scaling.factor"] 
(cfi.new<-1-(a/a.baseline*(1-cfi.old)))

## Display R^2 and fit measures

In [23]:
fitmeasures(WMHOB_fit) 
inspect(WMHOB_fit, 'r2') 